In [97]:
"""
You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This 


"""

    
def get_something(s: str) -> None:
    return
    
with open("input.txt") as f:
    print(get_something(f.read()))
    
# total is 

6951
